## Cross Elasticity (compare different products):

* One of the most important things when you are setting a price or a promotion, it is to know with whom are you competing to, what other product or products promotions does affect our product sales. 
* When other products have a discount in their prices, how much these discounts affect our product sales $\rightarrow$ cross-elasticity

* Cross-elasticity:
  * Cross-price elasticities explains the sales demand changes of our product, when other products prices increase or decrease. In other words, it explains the influence of other product price changes towards our product sales demand;
  * **Cross-price elasticities allow us to identify price competitors/substitutes and complements.**

In [65]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
import zipfile
import re
import matplotlib.pyplot as plt
import seaborn as sns

### Load data from Kaggle

There is a bug on Kaggle lib. So I need a old version

In [49]:
# !kaggle datasets download -d datafiniti/electronic-products-prices
! kaggle datasets list -s 'datafiniti/electronic-products-prices'

! kaggle datasets download -d datafiniti/electronic-products-prices


ref                                    title                                 size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------  ------------------------------------  ----  -------------------  -------------  ---------  ---------------  
datafiniti/electronic-products-prices  Electronic Products and Pricing Data   9MB  2018-07-31 18:34:22           9829        150  0.7058824        
Dataset URL: https://www.kaggle.com/datasets/datafiniti/electronic-products-prices
License(s): CC-BY-NC-SA-4.0
electronic-products-prices.zip: Skipping, found more recently modified local copy (use --force to force download)


#### Unizp and store data

In [50]:
# Ensure the 'data' directory exists
os.makedirs("data", exist_ok=True)

# Corrected filename
zip_path = "electronic-products-prices.zip"  # Check the exact name
extract_path = "data"

# Unzip the file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

In [51]:
pricing_data = "./data/DatafinitiElectronicsProductsPricingData.csv"
df = pd.read_csv(pricing_data)

# Better view the df width
pd.options.display.width = 0

print("Dataframe Dimensions:", df.shape)

Dataframe Dimensions: (14592, 31)


In [52]:
df.columns

Index(['id', 'prices.amountMax', 'prices.amountMin', 'prices.availability',
       'prices.condition', 'prices.currency', 'prices.dateSeen',
       'prices.isSale', 'prices.merchant', 'prices.shipping',
       'prices.sourceURLs', 'asins', 'brand', 'categories', 'dateAdded',
       'dateUpdated', 'ean', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'name', 'primaryCategories', 'sourceURLs', 'upc',
       'weight', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30'],
      dtype='object')

### Drop unecessary columns

In [53]:
# Drop columns that are not going to be used for the project
df = df[
    df.columns.drop(
        [
            "sourceURLs",
            "prices.availability",
            "asins",
            "manufacturerNumber",
            "upc",
            "primaryCategories",
            "keys",
            "id",
            "ean",
        ]
    )
]
# Drop columns that started with Unnamed
df = df[df.columns.drop(list(df.filter(regex="Unnamed*")))]

print("Dataframe Dimensions:", df.shape)

Dataframe Dimensions: (14592, 17)


In [54]:
df.columns

Index(['prices.amountMax', 'prices.amountMin', 'prices.condition',
       'prices.currency', 'prices.dateSeen', 'prices.isSale',
       'prices.merchant', 'prices.shipping', 'prices.sourceURLs', 'brand',
       'categories', 'dateAdded', 'dateUpdated', 'imageURLs', 'manufacturer',
       'name', 'weight'],
      dtype='object')

#### Rename columns for better interpretation

In [55]:
# Change name of columns
columns_to_change = {}
for column in df.columns:
    if column.startswith("prices."):
        columns_to_change[column] = column[7:]

columns_to_change
df.rename(columns=columns_to_change, inplace=True)
df.rename(
    columns={
        "amountMax": "price",
        "amountMin": "disc_price",
        "categories": "p_description",
    },
    inplace=True,
)

df

,price,disc_price,condition,currency,dateSeen,isSale,merchant,shipping,sourceURLs,brand,p_description,dateAdded,dateUpdated,imageURLs,manufacturer,name,weight
0,104.99,104.99,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",False,Bestbuy.com,NaN,http://www.bestbuy.com/site/sanus-tv-wall-moun...,Sanus,"Audio & Video Accessories,TV Mounts,TV Accesso...",2015-04-13T12:00:51Z,2018-05-12T18:59:48Z,https://images-na.ssl-images-amazon.com/images...,NaN,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,32.8 pounds
1,69.00,64.99,New,USD,2017-12-14T06:00:00Z,True,Walmart.com,Expedited,https://www.walmart.com/ip/BOYTONE-BT210FB-BLA...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,14 pounds
2,69.00,69.00,New,USD,2017-09-08T05:00:00Z,False,Walmart.com,Expedited,https://www.walmart.com/ip/Boytone-BT-210F-Blu...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,14 pounds
3,69.99,69.99,New,USD,2017-10-10T05:00:00Z,False,Bestbuy.com,NaN,https://www.bestbuy.com/site/boytone-2500w-2-1...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,14 pounds
4,66.99,66.99,New,USD,2017-08-28T07:00:00Z,False,Bestbuy.com,NaN,https://www.bestbuy.com/site/boytone-2500w-2-1...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,14 pounds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14587,65.99,65.99,NaN,USD,2015-09-05T00:00:00Z,True,NaN,USD 13.81 shipping,http://www.amazon.com/NAXA-NHS-7008-Bluetooth-...,naxa,"TVs & Electronics,Computers,Home Theater & Aud...",2015-09-11T02:03:11Z,2018-06-13T19:43:55Z,http://pisces.bbystatic.com/image2/BestBuy_US/...,Naxa,naxa - 2.1-Channel Soundbar with 50-Watt Digit...,4.1 pounds
14588,58.99,58.99,New,USD,"2017-10-10T19:00:00Z,2017-09-06T17:00:00Z,2017...",False,Bestbuy.com,NaN,https://www.bestbuy.com/site/naxa-2-1-channel-...,naxa,"TVs & Electronics,Computers,Home Theater & Aud...",2015-09-11T02:03:11Z,2018-06-13T19:43:55Z,http://pisces.bbystatic.com/image2/BestBuy_US/...,Naxa,naxa - 2.1-Channel Soundbar with 50-Watt Digit...,4.1 pounds
14589,58.49,58.49,New,USD,2018-03-05T11:00:00Z,False,Walmart.com,Standard,https://www.walmart.com/ip/NAXA-42-Sound-Bar-w...,naxa,"TVs & Electronics,Computers,Home Theater & Aud...",2015-09-11T02:03:11Z,2018-06-13T19:43:55Z,http://pisces.bbystatic.com/image2/BestBuy_US/...,Naxa,naxa - 2.1-Channel Soundbar with 50-Watt Digit...,4.1 pounds
14590,77.98,77.98,NaN,USD,2016-03-22T00:00:00Z,True,NaN,NaN,http://www.walmart.com/ip/Naxa-Nhs-7008-2.1-So...,naxa,"TVs & Electronics,Computers,Home Theater & Aud...",2015-09-11T02:03:11Z,2018-06-13T19:43:55Z,http://pisces.bbystatic.com/image2/BestBuy_US/...,Naxa,naxa - 2.1-Channel Soundbar with 50-Watt Digit...,4.1 pounds


In [56]:
### Evaluate isSale is correct

In [57]:
df.dtypes

price            float64
disc_price       float64
condition         object
currency          object
dateSeen          object
isSale              bool
merchant          object
shipping          object
sourceURLs        object
brand             object
p_description     object
dateAdded         object
dateUpdated       object
imageURLs         object
manufacturer      object
name              object
weight            object
dtype: object

In [ ]:
# Doublecheck Sale Prices
df["isSale"] = np.where(df["price"] > df["disc_price"], 1, 0)

In [61]:
df

,price,disc_price,condition,currency,dateSeen,isSale,merchant,shipping,sourceURLs,brand,p_description,dateAdded,dateUpdated,imageURLs,manufacturer,name,weight
0,104.99,104.99,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",0,Bestbuy.com,NaN,http://www.bestbuy.com/site/sanus-tv-wall-moun...,Sanus,"Audio & Video Accessories,TV Mounts,TV Accesso...",2015-04-13T12:00:51Z,2018-05-12T18:59:48Z,https://images-na.ssl-images-amazon.com/images...,NaN,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,32.8 pounds
1,69.00,64.99,New,USD,2017-12-14T06:00:00Z,1,Walmart.com,Expedited,https://www.walmart.com/ip/BOYTONE-BT210FB-BLA...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,14 pounds
2,69.00,69.00,New,USD,2017-09-08T05:00:00Z,0,Walmart.com,Expedited,https://www.walmart.com/ip/Boytone-BT-210F-Blu...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,14 pounds
3,69.99,69.99,New,USD,2017-10-10T05:00:00Z,0,Bestbuy.com,NaN,https://www.bestbuy.com/site/boytone-2500w-2-1...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,14 pounds
4,66.99,66.99,New,USD,2017-08-28T07:00:00Z,0,Bestbuy.com,NaN,https://www.bestbuy.com/site/boytone-2500w-2-1...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,14 pounds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14587,65.99,65.99,NaN,USD,2015-09-05T00:00:00Z,0,NaN,USD 13.81 shipping,http://www.amazon.com/NAXA-NHS-7008-Bluetooth-...,naxa,"TVs & Electronics,Computers,Home Theater & Aud...",2015-09-11T02:03:11Z,2018-06-13T19:43:55Z,http://pisces.bbystatic.com/image2/BestBuy_US/...,Naxa,naxa - 2.1-Channel Soundbar with 50-Watt Digit...,4.1 pounds
14588,58.99,58.99,New,USD,"2017-10-10T19:00:00Z,2017-09-06T17:00:00Z,2017...",0,Bestbuy.com,NaN,https://www.bestbuy.com/site/naxa-2-1-channel-...,naxa,"TVs & Electronics,Computers,Home Theater & Aud...",2015-09-11T02:03:11Z,2018-06-13T19:43:55Z,http://pisces.bbystatic.com/image2/BestBuy_US/...,Naxa,naxa - 2.1-Channel Soundbar with 50-Watt Digit...,4.1 pounds
14589,58.49,58.49,New,USD,2018-03-05T11:00:00Z,0,Walmart.com,Standard,https://www.walmart.com/ip/NAXA-42-Sound-Bar-w...,naxa,"TVs & Electronics,Computers,Home Theater & Aud...",2015-09-11T02:03:11Z,2018-06-13T19:43:55Z,http://pisces.bbystatic.com/image2/BestBuy_US/...,Naxa,naxa - 2.1-Channel Soundbar with 50-Watt Digit...,4.1 pounds
14590,77.98,77.98,NaN,USD,2016-03-22T00:00:00Z,0,NaN,NaN,http://www.walmart.com/ip/Naxa-Nhs-7008-2.1-So...,naxa,"TVs & Electronics,Computers,Home Theater & Aud...",2015-09-11T02:03:11Z,2018-06-13T19:43:55Z,http://pisces.bbystatic.com/image2/BestBuy_US/...,Naxa,naxa - 2.1-Channel Soundbar with 50-Watt Digit...,4.1 pounds


### Text Normalization - Product condition

In [ ]:
# Filtered Product Condition by Used, New and Refurbished
condition = df["condition"].fillna("None")

df["condition"] = np.where(
    condition.str.contains("new", flags=re.IGNORECASE),
    "New",
    np.where(
        condition.str.contains("refurbished", flags=re.IGNORECASE),
        "Refurbished",
        np.where(
            condition.str.contains("pre-owned|used", flags=re.IGNORECASE),
            "Used",
            condition,  # Default to original value if no match
        ),
    ),
)


c = df["condition"].value_counts()
print(
    "Product Condition Percentage\n{}\n{}\n\nProduct Condition values count\n{}\n{}".format(
        50 * "-", c / len(df.index) * 100, 50 * "-", c
    )
)

Product Condition Percentage
--------------------------------------------------
condition
New                         85.649671
None                         9.614857
Used                         3.125000
Refurbished                  1.576206
For parts or not working     0.034265
Name: count, dtype: float64

Product Condition values count
--------------------------------------------------
condition
New                         12498
None                         1403
Used                          456
Refurbished                   230
For parts or not working        5
Name: count, dtype: int64


condition
New                         12498
None                         1403
Used                          456
Refurbished                   230
For parts or not working        5
Name: count, dtype: int64

### Text Normalization - Merchant Website

In [74]:
# for the Merchants accuracy, we will use the source URL to include the correct merchant
# df['sourceURLs'][(df['merchant']== 'Not Specified') | (df['merchant']== 'Other')]
merchant_ab = df["sourceURLs"]
df["merchant"] = np.where(
    merchant_ab.str.contains("bhphotovideo", flags=re.IGNORECASE),
    "bhphotovideo.com",
    np.where(
        merchant_ab.str.contains("eBay|e bay", flags=re.IGNORECASE),
        "ebay.com",
        np.where(
            merchant_ab.str.contains("Amazon", flags=re.IGNORECASE),
            "Amazon.com",
            np.where(
                merchant_ab.str.contains("Bestbuy", flags=re.IGNORECASE),
                "Bestbuy.com",
                np.where(
                    merchant_ab.str.contains("Homedepot", flags=re.IGNORECASE),
                    "homedepot.com",
                    np.where(
                        merchant_ab.str.contains("newegg", flags=re.IGNORECASE),
                        "newegg.com",
                        np.where(
                            merchant_ab.str.contains("kmart", flags=re.IGNORECASE),
                            "kmart.com",
                            np.where(
                                merchant_ab.str.contains("frys", flags=re.IGNORECASE),
                                "frys.com",
                                np.where(
                                    merchant_ab.str.contains(
                                        "cdw", flags=re.IGNORECASE
                                    ),
                                    "cdw.com",
                                    np.where(
                                        merchant_ab.str.contains(
                                            "target", flags=re.IGNORECASE
                                        ),
                                        "target.com",
                                        np.where(
                                            merchant_ab.str.contains(
                                                "overstock", flags=re.IGNORECASE
                                            ),
                                            "overstock.com",
                                            np.where(
                                                merchant_ab.str.contains(
                                                    "barcodable", flags=re.IGNORECASE
                                                ),
                                                "barcodable.com",
                                                np.where(
                                                    merchant_ab.str.contains(
                                                        "kohls", flags=re.IGNORECASE
                                                    ),
                                                    "kohls.com",
                                                    np.where(
                                                        merchant_ab.str.contains(
                                                            "sears", flags=re.IGNORECASE
                                                        ),
                                                        "sears.com",
                                                        np.where(
                                                            merchant_ab.str.contains(
                                                                "Wal-mart|Walmart",
                                                                flags=re.IGNORECASE,
                                                            ),
                                                            "Walmart.com",
                                                            "Other",
                                                        ),
                                                    ),
                                                ),
                                            ),
                                        ),
                                    ),
                                ),
                            ),
                        ),
                    ),
                ),
            ),
        ),
    ),
)

In [ ]:
m = df["merchant"].value_counts()
print(
    "Merchant Percentage\n{}\n{}\n\nMerchant values count\n{}\n{}".format(
        50 * "-", m / len(df.index) * 100, 50 * "-", m
    )
)

Merchant Percentage
--------------------------------------------------
merchant
Bestbuy.com         30.838816
Walmart.com         25.479715
bhphotovideo.com    18.893914
ebay.com            14.624452
Amazon.com           7.147752
target.com           0.842928
frys.com             0.507127
barcodable.com       0.322094
newegg.com           0.308388
cdw.com              0.294682
Other                0.260417
homedepot.com        0.157621
kmart.com            0.123355
kohls.com            0.075384
sears.com            0.068531
overstock.com        0.054825
Name: count, dtype: float64

Merchant values count
--------------------------------------------------
merchant
Bestbuy.com         4500
Walmart.com         3718
bhphotovideo.com    2757
ebay.com            2134
Amazon.com          1043
target.com           123
frys.com              74
barcodable.com        47
newegg.com            45
cdw.com               43
Other                 38
homedepot.com         23
kmart.com             18
kohl

### Set dates format

Data shows the date that an advertisment with specific price was seen, often named this action in tech companies and marketing as impressions.

df